In [1]:
import dlib as dl
from glob import glob
import os

face_detector = dl.get_frontal_face_detector()

shape_predictor = dl.shape_predictor(
    "/Education/imageProcessing/imageProcessingLabs/Lab4/models/shape_predictor_68_face_landmarks.dat"
)

face_encoder = dl.face_recognition_model_v1(
    "/Education/imageProcessing/imageProcessingLabs/Lab4/models/dlib_face_recognition_resnet_model_v1.dat"
)

VALID_EXTENSIONS = ['.png','.jpg','.jpeg']

def get_image_paths(root_dir, class_names):
    image_paths = []
    for class_name in class_names:
        class_dir = os.path.sep.join([root_dir,class_name])
        class_file_paths = glob(os.path.sep.join([class_dir,'*.*']))


        for file_path in class_file_paths:
            ext = os.path.splitext(file_path)[1]
            if ext.lower() not in VALID_EXTENSIONS:
                print("Skippinng file: " + file_path)
                continue

            image_paths.append(file_path)
             
    return image_paths    


In [2]:
import dlib

class_names = ['Danil_Smyslov']
image_paths = get_image_paths("dataset",class_names)

print(image_paths)

['dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-00.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-01.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-02 (2).jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-02.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-03 (2).jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-03.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-04.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-05.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-06 (2).jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-06.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-07 (2).jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-07.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-08.jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-09 (2).jpg', 'dataset\\Danil_Smyslov\\photo_2023-03-01_12-41-09.jpg']


In [3]:
import cv2
def face_rects(image):
    grayImg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = face_detector(grayImg,1)
    return rects

In [4]:
def face_landmarks(image):
    return [shape_predictor(image, face_rect)
            for face_rect in face_rects(image)]

In [5]:
def face_encodings(image):
    return [np.array(face_encoder.compute_face_descriptor(image, face_landmark))
            for face_landmark in face_landmarks(image)]

In [6]:
import numpy as np
root_dir = "dataset"

class_names = os.listdir(root_dir)

image_paths = get_image_paths(root_dir,class_names)

name_encodings_dict = {}
print(image_paths)


['dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0001.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0002.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0003.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0004.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0005.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0006.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0007.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0008.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0009.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0010.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0011.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0012.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0013.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0014.jpg', 'dataset\\Arnold_Schwarzenegger\\Arnold_Schwarzenegger_0015.jpg', 'dataset\

In [7]:
import pickle 
nb_current_image = 1

for image_path in image_paths:
    print("Image processed " + str(nb_current_image) + "/" + str(len(image_paths)))
    image = cv2.imread(image_path)
    encodings = face_encodings(image)
    name = image_path.split(os.path.sep)[-2]
    e = name_encodings_dict.get(name,[])
    e.extend(encodings)
    name_encodings_dict[name] = e
    nb_current_image += 1

with open("encodings.pickle","wb") as f:
    pickle.dump(name_encodings_dict,f)

Image processed 1/57
Image processed 2/57
Image processed 3/57
Image processed 4/57
Image processed 5/57
Image processed 6/57
Image processed 7/57
Image processed 8/57
Image processed 9/57
Image processed 10/57
Image processed 11/57
Image processed 12/57
Image processed 13/57
Image processed 14/57
Image processed 15/57
Image processed 16/57
Image processed 17/57
Image processed 18/57
Image processed 19/57
Image processed 20/57
Image processed 21/57
Image processed 22/57
Image processed 23/57
Image processed 24/57
Image processed 25/57
Image processed 26/57
Image processed 27/57
Image processed 28/57
Image processed 29/57
Image processed 30/57
Image processed 31/57
Image processed 32/57
Image processed 33/57
Image processed 34/57
Image processed 35/57
Image processed 36/57
Image processed 37/57
Image processed 38/57
Image processed 39/57
Image processed 40/57
Image processed 41/57
Image processed 42/57
Image processed 43/57
Image processed 44/57
Image processed 45/57
Image processed 46/

In [8]:
with open("encodings.pickle", "rb") as f:
    name_encodings_dict = pickle.load(f)
image = cv2.imread("/Education/imageProcessing/imageProcessingLabs/Lab4/examples/3.jpg")
encodings = face_encodings(image)
names = []

In [9]:
def nb_of_matches(known_encodings, unknown_encoding):
    distances = np.linalg.norm(known_encodings - unknown_encoding,axis=1)
    small_distances = distances <= 0.6
    return sum(small_distances)

In [10]:
for encoding in encodings:
    counts = {}
    for(name,encodings) in name_encodings_dict.items():
        counts[name] = nb_of_matches(encodings, encoding)
    if all(count == 0 for count in counts.values()):
        name = "Unknown"
    else:
        name = max(counts, key = counts.get)
    names.append(name)

In [11]:
for rect,name in zip(face_rects(image),names):
    x1,y1,x2,y2 = rect.left(),rect.top(),rect.right(),rect.bottom()
    cv2.rectangle(image,(x1,y1),(x2,y2),(0,0,255),2)
    cv2.putText(image,name,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.72,(0,0,255),2)

cv2.imshow("image",image)
cv2.waitKey(0)

-1